<a href="https://colab.research.google.com/github/jbeen2/NH-FakeNews/blob/main/Code/Train_HuggingFace_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

import re

import os 
import json
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm, tnrange, tqdm_notebook
import time
import datetime

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.5MB 5.9MB/s 
     |████████████████████████████████| 890kB 24.0MB/s 
     |████████████████████████████████| 2.9MB 46.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=d4cd4deb62079b2b8e2db00a1151f4bb80efe0b1724320d8a216ff987d8f83ec
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla V100-SXM2-16GB'

# Data Load 

In [ ]:
cd /content/drive/MyDrive/Projects/NH_dacon/재빈

/content/drive/MyDrive/Projects/NH_dacon/재빈


In [ ]:
train = pd.read_csv(os.path.join("../Data", "news_train.csv"))
test = pd.read_csv(os.path.join("../Data", "news_test.csv"))
submission = pd.read_csv(os.path.join("../Data", "sample_submission.csv"))

print(train.shape, test.shape, submission.shape)

(118745, 6) (142565, 6) (142565, 2)


In [ ]:
train.head()

,n_id,date,title,content,ord,info
0,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도,1,0
1,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"""실적기반"" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개",2,1
2,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시",3,1
3,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지,4,0
4,NEWS09727,20200626,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참",전국적인 소비 붐 조성에 기여할 예정,1,0


In [ ]:
def clean_text(text) : 
    text = re.sub('↑', '상승 ', text)
    text = re.sub('↓', '하락 ', text)
    text = re.sub('㈜', '주식회사 ', text)
    return text

In [ ]:
# preprocessing 

train["title"] = train.apply(lambda x : clean_text(x["title"]), axis=1)
train["content"] = train.apply(lambda x : clean_text(x["content"]), axis=1)
train.head()

,n_id,date,title,content,ord,info
0,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도,1,0
1,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"""실적기반"" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개",2,1
2,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시",3,1
3,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지,4,0
4,NEWS09727,20200626,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참",전국적인 소비 붐 조성에 기여할 예정,1,0


In [ ]:
test["title"] = test.apply(lambda x : clean_text(x["title"]), axis=1)
test["content"] = test.apply(lambda x : clean_text(x["content"]), axis=1)
test.head()

,n_id,date,title,content,ord,id
0,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",마이크로 LED TV 장비 양산 돌입- 전방업체 투자 확대로 본업도 호조연일 '신고가',1,NEWS00237_1
1,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",[이데일리 김대웅 기자] 반도체 장비 업체 코세스(089890)의 기술력이 마이크로...,2,NEWS00237_2
2,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",최근 대형 업체들과 거래를 맺고 관련 장비들의 양산에 돌입하면서 주가도 연일 신고가...,3,NEWS00237_3
3,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",TV를 필두로 올해부터 마이크로 LED의 시대가 본격적으로 개화할 것으로 예상되면서...,4,NEWS00237_4
4,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대","코세스는 반도체 장비를 제조, 판매하는 업체로 지난 2006년 11월 코스닥 시장에...",5,NEWS00237_5


In [ ]:
train_title = train["title"]
test_title = test["title"]

In [ ]:
train_content = train["content"]
test_content = test["content"]

In [ ]:
# BERT의 입력 형식에 맞게 변환
train_title = ["[CLS] " + str(sentence) + " [SEP] " for sentence in train_title]
test_title = ["[CLS] " + str(sentence) + " [SEP] " for sentence in test_title]

In [ ]:
train_text = [title + content + " [SEP]" for title, content in zip(train_title, train_content)]
test_text = [title + content + " [SEP]" for title, content in zip(test_title, test_content)]

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

In [ ]:
start = time.time()

train_tokenized_texts = [tokenizer.tokenize(sent) for sent in train_text]
test_tokenized_texts = [tokenizer.tokenize(sent) for sent in test_text]

print("tokenize takes : ", time.time() - start, "sec" )
print("=====" * 25)
print(train_text[0])
print(train_tokenized_texts[0])

tokenize takes :  91.17087030410767 sec
[CLS] [마감]코스닥 기관 678억 순매도 [SEP] [이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도 [SEP]
['[CLS]', '[', '마', '##감', ']', '코', '##스', '##닥', '기', '##관', '678', '##억', '순', '##매', '##도', '[SEP]', '[', '이', '##데', '##일', '##리', 'MA', '##R', '##K', '##ET', '##PO', '##IN', '##T', ']', '15', ':', '32', '현재', '코', '##스', '##닥', '기', '##관', '678', '##억', '순', '##매', '##도', '[SEP]']


In [ ]:
print(test_text[0])
print(test_tokenized_texts[0])

[CLS] [주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대 [SEP] 마이크로 LED TV 장비 양산 돌입- 전방업체 투자 확대로 본업도 호조연일 '신고가' [SEP]
['[CLS]', '[', '주', '##목', '!', 'e', '##스', '##몰', '##캡', ']', '코', '##세', '##스', ',', '마', '##이', '##크', '##로', '##LE', '##D', '시', '##장', '개', '##화', '최대', '##수', '##혜', '기', '##대', '[SEP]', '마', '##이', '##크', '##로', 'LED', 'TV', '장', '##비', '양', '##산', '돌', '##입', '-', '전', '##방', '##업', '##체', '투', '##자', '확', '##대로', '본', '##업', '##도', '호', '##조', '##연', '##일', "'", '신', '##고', '##가', "'", '[SEP]']


In [ ]:
# train 길이 분포 출력 
tkn_len = [len(s) for s in train_tokenized_texts]

print("title 최소 길이 : ", np.min(tkn_len))
print("title 최대 길이 : ", np.max(tkn_len))
print("title 평균 길이 : ", np.mean(tkn_len))

title 최소 길이 :  12
title 최대 길이 :  2356
title 평균 길이 :  60.184858309823575


In [ ]:
def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if len(s) <= max_len:
            cnt = cnt + 1
    print("전체 샘플 중 길이가 {} 이하인 샘플의 비율 : {}".format(max_len, (cnt/len(nested_list))))

In [ ]:
below_threshold_len(128, train_tokenized_texts)

전체 샘플 중 길이가 128 이하인 샘플의 비율 : 0.9845551391637543


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
train_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in train_tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post", value=1)

train_input_ids[0]

array([   101,    164,   9246, 105197,    166,   9812,  12605, 118770,
         8932,  20595,  69015,  91837,   9462, 100372,  12092,    102,
          164,   9638,  28911,  18392,  12692,  27277,  11273,  11733,
        52338,  93520,  27128,  11090,    166,  10208,    131,  10842,
        26565,   9812,  12605, 118770,   8932,  20595,  69015,  91837,
         9462, 100372,  12092,    102,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
      

In [ ]:
test_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in test_tokenized_texts]
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post", value=1)
test_input_ids[0]

array([   101,    164,   9689,  68055,    106,    173,  12605, 118937,
       119295,    166,   9812,  24982,  12605,    117,   9246,  10739,
        20308,  11261,  51036,  11490,   9485,  13890,   8857,  18227,
        99405,  15891, 119437,   8932,  14423,    102,   9246,  10739,
        20308,  11261,  51829,  10813,   9657,  29455,   9543,  21386,
         9091,  58303,    118,   9665,  42337,  26784,  29683,   9881,
        13764,   9994,  37601,   9358,  26784,  12092,   9985,  20626,
        25486,  18392,    112,   9487,  11664,  11287,    112,    102,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
      

In [ ]:
# train 어텐션 마스크 초기화
train_attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in train_input_ids:
    seq_mask = [float(i != 1) for i in seq]
    train_attention_masks.append(seq_mask)

print(train_attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# test 어텐션 마스크 초기화
test_attention_masks = []

for seq in test_input_ids:
    seq_mask = [float(i != 1) for i in seq]
    test_attention_masks.append(seq_mask)

print(test_attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# label
labels = np.array(train['info'])

# token id 
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(train_input_ids, labels, 
                                                                   random_state=2020, test_size=0.2)

# attention mask 
train_masks, validation_masks, _, _ = train_test_split(train_attention_masks, train_content,
                                                       random_state=2020, test_size=0.2)

In [ ]:
train_inputs.shape, validation_inputs.shape, train_labels.shape, validation_labels.shape

((94996, 128), (23749, 128), (94996,), (23749,))

In [ ]:
# to tensor 
train_inputs, train_labels = torch.tensor(train_inputs) , torch.tensor(train_labels)
validation_inputs, validation_labels = torch.tensor(validation_inputs) , torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
print("*** BERT input ***")
print("=====" * 20)
print("* train input")
print(train_inputs[0])
print()
print("* train label")
print(train_labels[0])
print()
print("* train attention mask")
print(train_masks[0])
print()
print()
print("* validation input")
print(validation_inputs[0])
print()
print("* validation label")
print(validation_labels[0])
print()
print("* validation attention mask")
print(validation_masks[0])

*** BERT input ***
* train input
tensor([   101,   2650,    117,   9309,  37388,  43022,  33188,  21928,   9736,
         21928,  10530,    100,   9356,  15891,   9098, 106249,  27654,   9657,
         40419,  88236,   8987,  20626,  18471,   9067,  20479,    100,    102,
          9689,  21155,  14040,  83685,   9566,  32159,  25805,   9513,   9566,
         14423,  37568,   9684,  68055,    119,   9284,  37824,  17322,    110,
         66982,    119,    102,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
       

In [ ]:
# batch size 설정
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)     # [input, mask, label]
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
next(iter(train_dataloader))

[tensor([[   101,   8324,    107,  ...,      1,      1,      1],
         [   101,   9992,  37004,  ...,      1,      1,      1],
         [   101,   9580, 118762,  ...,      1,      1,      1],
         ...,
         [   101,   9407,  26784,  ...,      1,      1,      1],
         [   101,   9670,  14646,  ...,      1,      1,      1],
         [   101,   9434, 119182,  ...,      1,      1,      1]]),
 tensor([[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.]]),
 tensor([0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
         1, 0, 0, 1, 0, 1, 0, 0])]

In [ ]:
a, b, c = next(iter(train_dataloader))
print("* shape *")
print(a.shape)
print(b.shape)
print(c.shape)
print()
print("* type *")
print(type(a))
print(type(b))
print(type(c))

* shape *
torch.Size([32, 128])
torch.Size([32, 128])
torch.Size([32])

* type *
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>


# Modeling 

In [ ]:
# https://huggingface.co/models
# https://huggingface.co/monologg/kobert#how-to-use

model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# Optimizer
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# epoch
epochs = 5

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))     # hh:mm:ss

In [ ]:
model.zero_grad() 

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0

    model.train()
 
    for step, batch in enumerate(train_dataloader):
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

        loss = outputs[0]
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.5f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")
    t0 = time.time()

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():     
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
            
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.5f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

    output_file = "1230_multilingual/1230_bert_{}.pt".format(epoch_i + 1)
    torch.save(model.state_dict(), output_file)

print("")
print("Training complete!")


======== Epoch 1 / 5 ========
Training...
  Batch   500  of  2,969.    Elapsed: 0:03:19.
  Batch 1,000  of  2,969.    Elapsed: 0:06:37.
  Batch 1,500  of  2,969.    Elapsed: 0:09:54.
  Batch 2,000  of  2,969.    Elapsed: 0:13:12.
  Batch 2,500  of  2,969.    Elapsed: 0:16:30.

  Average training loss: 0.05471
  Training epoch took: 0:19:36

Running Validation...
  Accuracy: 0.99251
  Validation took: 0:01:31

======== Epoch 2 / 5 ========
Training...
  Batch   500  of  2,969.    Elapsed: 0:03:18.
  Batch 1,000  of  2,969.    Elapsed: 0:06:35.
  Batch 1,500  of  2,969.    Elapsed: 0:09:52.
  Batch 2,000  of  2,969.    Elapsed: 0:13:10.
  Batch 2,500  of  2,969.    Elapsed: 0:16:27.

  Average training loss: 0.01900
  Training epoch took: 0:19:33

Running Validation...
  Accuracy: 0.99378
  Validation took: 0:01:31

======== Epoch 3 / 5 ========
Training...
  Batch   500  of  2,969.    Elapsed: 0:03:18.
  Batch 1,000  of  2,969.    Elapsed: 0:06:35.
  Batch 1,500  of  2,969.    Elapsed:

### 추가학습 +2

In [ ]:
model.load_state_dict(torch.load("1230_multilingual/1230_bert_5.pt"), strict=False)

<All keys matched successfully>

In [ ]:
epochs = 2

In [ ]:
model.zero_grad() 

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 5 + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0

    model.train()
 
    for step, batch in enumerate(train_dataloader):
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

        loss = outputs[0]
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.5f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")
    t0 = time.time()

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():     
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
            
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.5f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

    output_file = "1230_multilingual/1230_bert_{}.pt".format(epoch_i + 5 + 1)
    torch.save(model.state_dict(), output_file)

print("")
print("Training complete!")


======== Epoch 6 / 7 ========
Training...
  Batch   500  of  2,969.    Elapsed: 0:03:17.
  Batch 1,000  of  2,969.    Elapsed: 0:06:34.
  Batch 1,500  of  2,969.    Elapsed: 0:09:52.
  Batch 2,000  of  2,969.    Elapsed: 0:13:09.
  Batch 2,500  of  2,969.    Elapsed: 0:16:26.

  Average training loss: 0.00051
  Training epoch took: 0:19:30

Running Validation...
  Accuracy: 0.99596
  Validation took: 0:01:31

======== Epoch 7 / 7 ========
Training...
  Batch   500  of  2,969.    Elapsed: 0:03:17.
  Batch 1,000  of  2,969.    Elapsed: 0:06:35.
  Batch 1,500  of  2,969.    Elapsed: 0:09:52.
  Batch 2,000  of  2,969.    Elapsed: 0:13:09.
  Batch 2,500  of  2,969.    Elapsed: 0:16:26.

  Average training loss: 0.00051
  Training epoch took: 0:19:31

Running Validation...
  Accuracy: 0.99596
  Validation took: 0:01:31

Training complete!


In [ ]:
model.load_state_dict(torch.load("1230_multilingual/1230_bert_7.pt"), strict=False)

<All keys matched successfully>

In [ ]:
# torch 모델 저장
model.save_pretrained('/content/drive/MyDrive/Projects/NH_dacon/재빈/final')

# Test

In [ ]:
# to tensor
test_inputs = torch.tensor(test_input_ids)
test_masks = torch.tensor(test_attention_masks)

In [ ]:
test_inputs.shape, test_masks.shape

(torch.Size([142565, 128]), torch.Size([142565, 128]))

In [ ]:
test_data = TensorDataset(test_inputs, test_masks)
test_dataloader = DataLoader(test_data, batch_size = 32)

In [ ]:
# ========================================
#                Predict
# ========================================

print("")
print("Test...")

t0 = time.time()

model.eval()

predictions = [] 

for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask = batch

    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    predictions.append(logits)

print("")
print("Test took: {:}".format(format_time(time.time() - t0)))
print("Test complete!")


Test...

Test took: 0:09:08
Test complete!


In [ ]:
result = [] 
for i in range(len(predictions)) : 
    result.extend([np.argmax(pred) for pred in predictions[i]])

In [ ]:
submission["info"] = result 

In [ ]:
submission["info"].value_counts()

0    88258
1    54307
Name: info, dtype: int64

# 덧씌우기

In [ ]:
train_unique_ad_sentence = train.query('info == "1"')['content'].unique()
train_unique_real_sentence = train.query('info == "0"')['content'].unique()
test_unique_sentence = test['content'].unique()

In [ ]:
test_content = test['content'].values

for idx, sent in enumerate(tqdm_notebook(test_content)) : # Test 데이터에 있는 모든 content들에 대하여

    if sent in train_unique_ad_sentence:     # Train 데이터의 광고성 문구와 같은지 비교
        submission['info'].iloc[idx] = 1     # 같으면 1

    if sent in train_unique_real_sentence:   # Train 데이터의 진짜 뉴스 문구와 같은지 비교
        submission['info'].iloc[idx] = 0     # 같으면 0

print(submission.head())


            id  info
0  NEWS00237_1     1
1  NEWS00237_2     0
2  NEWS00237_3     0
3  NEWS00237_4     0
4  NEWS00237_5     0


In [ ]:
submission["info"].value_counts()

0    88374
1    54191
Name: info, dtype: int64

In [ ]:
submission.to_csv(os.path.join('submission', "1230_multilingual.csv"), index=False)